## Импорт необходимых библиотек

In [ ]:
!pip install -U gensim --user

In [ ]:
!pip install tweet-preprocessor kaggle scikit-learn

In [17]:
import pandas as pd
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import gensim
from sklearn.model_selection import train_test_split
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

ModuleNotFoundError: No module named 'gensim'

## Анализ данных

In [ ]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files('kazanova/sentiment140')

In [ ]:
with zipfile.ZipFile('sentiment140.zip', 'r') as zipref:
    zipref.extractall('sentiment140/')

In [ ]:
colnames=['target', 'ids', 'date', 'flag', 'user', 'text']
data = pd.read_csv("sentiment140/training.1600000.processed.noemoticon.csv", encoding='cp1252', names=colnames, header=None)
data.sample(10)

In [ ]:
data.info()

In [ ]:
data.drop(['ids', 'date', 'flag', 'user'], axis=1, inplace=True)
data.head()

In [ ]:
data['target'].value_counts().plot(
    kind='bar',
    rot=0,
    title='Количество твитов по классам',
    xlabel='Класс (0 - отрицательные, 2 - нейтральные, 4 - позитивные',
    ylabel='Число твитов',
    legend=True,
    grid=True
);

Класса получается всего 2, поэтому мы будем делать бинарную классификацию. Заменим метку класса 4 на 1:

In [ ]:
data['target'] = data['target'].replace(4,1)

## Предобработка данных

In [ ]:
print(STOPWORDS)

Данные стоп слова вряд ли влияют на настроение твита, так что возьму их прямо из коробки.

In [ ]:
%%time
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    text = remove_stopwords(text)
    return text.lower().replace('\s\s+', ' ')

data['cleaned_text'] = data.apply(preprocess_tweet, axis=1)

In [ ]:
data.sample(10)

## Извлечение признаков из текста с помощью векторизации

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['target'], random_state=2023)

In [ ]:
%%time
VECTOR_SIZE = 256
model = gensim.models.FastText(
    sentences=X_train,
    vector_size=VECTOR_SIZE, # default = 100
    window=7, # default = 5
    min_count=10,
    sg=1, # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # Number of iterations (epochs) over the corpus
    seed=2023,
)